In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
from google.colab import drive

# Mounts Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Za dotreniranje sem uporabila https://github.com/msn199959/Logo-2k-plus-Dataset.

zip_file_path = '/content/drive/MyDrive/itap_koncni_zapiski/Logo-2k+/train_and_test.zip'


extract_path = '/content/drive/MyDrive/aixpravo'

if not os.path.exists(extract_path):
    os.makedirs(extract_path)


!unzip -q "{zip_file_path}" -d "{extract_path}"

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.RandomCrop((448, 448)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])
test_transform = transforms.Compose([
    transforms.Resize((600, 600), Image.BILINEAR),
    transforms.CenterCrop((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])

train_dataset = ImageFolder('/content/drive/MyDrive/aixpravo/train_and_test/train', transform=train_transform)
test_dataset = ImageFolder('/content/drive/MyDrive/aixpravo/train_and_test/test', transform=test_transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [4]:
# Naredimo funkcijo, ki bo natrenirala naš model (vzeto iz vaj ITAP)
def train(epochs, model, trainset, batch_size, device, learning_rate=0.01):

    # Najprej definiramo objekt s katerim bomo vzorčili podatke. Nevronske mreže se ponavadi uči iterativno, več epoh
    # in znotraj vsake epohe več iteracij
    # V vsaki iteraciji se učimo iz ene skupine podatkov (ang. batch)
    trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)

    # Zagotovimo, da bo model v načinu treniranja, kjer se računajo gradienti in so aktivni vsi sloji
    # (med evalvacijo niso nujno vsi aktivni, npr. Dropout, BatchNormalization, ...)
    model.train()

    # Definiramo naš optimizator. V našem primeru bo to stohastični gradientni spust (SGD) z learning rate-om 0.01
    # Lahko bi uporabili tudi Adam naprimer
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # Definiramo našo funkcijo izgube (loss). V našem primeru bo to prečna entropija.
    criterion = nn.CrossEntropyLoss()

    # Gremo čež epohe
    for epoch in range(epochs):

        running_loss = 0.0
        # Uporabimo funkcijo tqdm.tqdm, ki nam bo lepše, v realnem času izpisovala napredek učenja
        with tqdm(total=len(trainloader)*batch_size, desc=f'Training - Epoch: {epoch + 1}/{epochs}', unit='chunks') as prog_bar:
            # Gremo čez vse podatke v skupinah po batch_size z trainloaderjem, v našem primeru je to 32
            for i, data in enumerate(trainloader, 0):
                # Podatke razpakiramo v vhode in izhode (labele)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)


                # Počistimo (resetiramo) gradiente v podatkih
                optimizer.zero_grad()

                # Vhodne podatke spustimo čez model, ta nam vrne matriko, v kateri se vsaka vrstica sešteje v 1 (zaradi Softmax sloja)
                outputs = model(inputs)
                # Izračunamo izgubo
                loss = criterion(outputs, labels)
                # Naredimo vzvratno razširanje napake (backpropagation)
                loss.backward()
                # Naredimo en korak optimizacije
                optimizer.step()

                # Dodamo izgubo k naši vsoti izgube. S funkcijo detach poskrbimo, da ne prištejemo (in si tako shranimo) tudi gradienta
                # s funkcijo item() pa da se le vrednost in ne celoten vektor
                running_loss += loss.detach().item()

                # Posodobimo vrednosti funkcije tqdm.tqdm. Vsoto dosedanje izgube delimo s številom skupin, ki smo jih že obdelali
                prog_bar.set_postfix(**{'loss': (running_loss) / (i+1)})
                # Posodobimo progress bar
                prog_bar.update(batch_size)

    print('Finished Training')

In [5]:
from torchvision.models import resnet18, ResNet18_Weights

In [6]:
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V2)

features_dim = model.fc.in_features
model.fc = nn.Linear(features_dim, 2341)
#device = torch.device('/device:GPU:0')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 237MB/s]


In [7]:
print(device)

cuda


In [8]:
EPOCHS=5
BATCH_SIZE=32
train(EPOCHS, model, train_dataset, BATCH_SIZE, device)

Training - Epoch: 1/5:  11%|█         | 12704/116992 [2:59:21<24:32:25,  1.18chunks/s, loss=2.6]


KeyboardInterrupt: 

In [9]:
#Z uporabo celega train dataset-a od Logo-2k+ je šlo prepočasi(glede na rok za oddajo ideje) za to sem zmanjšala število epoh,
# povečala batch_size in za treniranje vzela test dataset iz Logo-2k+(znatno manjše število slik)

from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2
)

# (using ResNet18 for speed)
from torchvision import models
model = models.resnet18(pretrained=True)
num_classes = len(test_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Moved to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# fewer epochs
num_epochs = 2


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(test_loader)}')

# Saves the model
import os
save_path = '/content/drive/MyDrive/logo_model_project'
os.makedirs(save_path, exist_ok=True)

# Saves model state dictionary to Drive
torch.save(model.state_dict(), f'{save_path}/logo_model_resnet18.pth')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 213MB/s]


Epoch 1/2, Loss: 1.932428915485455
Epoch 2/2, Loss: 1.8831182065283416


In [11]:
# Downloading the files to my comp
import json


with open(f'{save_path}/logo_classes.json', 'w') as f:
    json.dump(test_dataset.class_to_idx, f)

from google.colab import files
files.download(f'{save_path}/logo_model_resnet18.pth')
files.download(f'{save_path}/logo_classes.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>